In [90]:
import pickle
import os
import pandas as pd
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)

import scripts.OrderBookAnalizer as oba
import scripts.functions as fn

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


## Revision del ultimo conjunto de datos obtenido

In [91]:
symbol='BTCUSDT'
LOG_DIR = os.path.join(BASE_DIR,'log')
DATA_FILE = os.path.join(LOG_DIR, f"order_book_{symbol}.pkl")
if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "rb") as archivo:
        depth = pickle.load(archivo)

analizer = oba.OrderBookAnalyzer()
record = analizer.analyze_order_book(depth['bids'], depth['asks'])
for r in record: #
    #print(r['price_group'],' - ',round(r['volume_pct'],1))
    print(r)


timestamp
base_price
bid_supports
ask_resistances
total_bid_volume
total_ask_volume
bid_volume_distribution
ask_volume_distribution
market_imbalance
price_scale_used


## Revision del historico

In [92]:
symbol='BTCUSDT'
LOG_DIR = os.path.join(BASE_DIR,'log')
DATA_FILE = os.path.join(LOG_DIR, f"order_book_{symbol}_historic.pkl")
if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "rb") as archivo:
        df = pickle.load(archivo)

## Analisis de los resultados

In [93]:
qty_recs = df['timestamp'].count()
if qty_recs>48:
    qty_recs = 8
print(f'Total de registros analizados: {qty_recs} - {(qty_recs*0.5)} Hs.')
print('Tiempo de datos almacenados: ',df.iloc[-1]['timestamp']-df.iloc[0]['timestamp'])

#Primer analisis sobre ultima lectura de ordenes
analysis_result = df.iloc[-1]
imbalance = analysis_result['market_imbalance']['imbalance_pct']

if imbalance > 25:
    print("Fuerte presión COMPRADORA - Posible movimiento alcista")
elif imbalance > 10:
    print("Moderada presión compradora")
elif imbalance < -25:
    print("Fuerte presión VENDEDORA - Posible movimiento bajista")
elif imbalance < -10:
    print("Moderada presión vendedora")
else:
    print("Mercado equilibrado")

analizer = oba.OrderBookAnalyzer(DATA_FILE)
summary = analizer.get_summary_stats(qty_recs)

print('high_price:\t', summary['high_price'])
print('mean_price:\t', summary['mean_price'])
print('low_price:\t', summary['low_price'])

print("Soportes clave históricos:")
for level in summary['support_levels']:
    print(f"• ${level['mean_price']:.2f}: "
          f"{level['mean_volume_pct']:.2f}% volumen "
          f"(apareció {level['frequency']} veces)")
    
print("Resistencias clave históricos:")
for level in summary['resistance_levels']:
    print(f"• ${level['mean_price']:.2f}: "
          f"{level['mean_volume_pct']:.2f}% volumen "
          f"(apareció {level['frequency']} veces)")
    
change = summary['price_change_pct']
direction = "subió" if change > 0 else "bajó"
print(f"El precio {direction} un {abs(change):.2f}% en las últimas {qty_recs/2} horas")


Total de registros analizados: 8 - 4.0 Hs.
Tiempo de datos almacenados:  6 days 21:30:00.862947
Mercado equilibrado
high_price:	 84572.005
mean_price:	 80901.53137048193
low_price:	 74718.655
Soportes clave históricos:
• $70000.00: 12.96% volumen (apareció 7 veces)
• $69000.00: 7.40% volumen (apareció 2 veces)
• $72007.33: 7.13% volumen (apareció 6 veces)
• $74009.65: 6.86% volumen (apareció 7 veces)
• $69048.00: 6.59% volumen (apareció 2 veces)
• $65019.24: 6.29% volumen (apareció 5 veces)
• $73003.84: 6.25% volumen (apareció 7 veces)
• $68000.00: 6.08% volumen (apareció 5 veces)
• $75052.65: 6.07% volumen (apareció 7 veces)
• $71000.00: 5.66% volumen (apareció 4 veces)
Resistencias clave históricos:
• $120000.00: 16.52% volumen (apareció 2 veces)
• $89395.70: 10.01% volumen (apareció 10 veces)
• $84784.94: 9.05% volumen (apareció 22 veces)
• $101418.97: 8.88% volumen (apareció 7 veces)
• $94989.71: 8.78% volumen (apareció 5 veces)
El precio subió un 0.44% en las últimas 4.0 horas


In [94]:
analizer.analyze_summary(summary,qty_recs)


=== Análisis de los últimos 4.0 horas ===

Tendencia del mercado: Presión COMPRADORA (4.76% de desbalance)

Soportes más consistentes:
- $70000: 7 apariciones (Vol: 13.0%)
- $74010: 7 apariciones (Vol: 6.9%)
- $73004: 7 apariciones (Vol: 6.3%)

Resistencias más consistentes:
- $84785: 22 apariciones (Vol: 9.1%)
- $89396: 10 apariciones (Vol: 10.0%)
- $101419: 7 apariciones (Vol: 8.9%)

Divergencia detectada entre precio y volumen


In [95]:
print(f"\n📈 Resumen {qty_recs/2}h | Δ {summary['price_change_pct']:+.2f}%")
print(f"⚖️ Desbalance: {summary['mean_imbalance']:+.1f}%")
print(f"🛡️ Soportes clave: {len(summary['support_levels'])}")
print(f"🚧 Resistencias: {len(summary['resistance_levels'])}")

print("\n🔝 Top resistencias:")
for s in sorted(summary['resistance_levels'], key=lambda x: -x['mean_volume_pct'])[:3]:
    print(f"• ${s['mean_price']:.0f} | 📊 {s['mean_volume_pct']:.1f}% | 🔁 {s['frequency']}x")

print("\n🔝 Top soportes:")
for s in sorted(summary['support_levels'], key=lambda x: -x['mean_volume_pct'])[:3]:
    print(f"• ${s['mean_price']:.0f} | 📊 {s['mean_volume_pct']:.1f}% | 🔁 {s['frequency']}x")


📈 Resumen 4.0h | Δ +0.44%
⚖️ Desbalance: +4.8%
🛡️ Soportes clave: 10
🚧 Resistencias: 5

🔝 Top resistencias:
• $120000 | 📊 16.5% | 🔁 2x
• $89396 | 📊 10.0% | 🔁 10x
• $84785 | 📊 9.1% | 🔁 22x

🔝 Top soportes:
• $70000 | 📊 13.0% | 🔁 7x
• $69000 | 📊 7.4% | 🔁 2x
• $72007 | 📊 7.1% | 🔁 6x


## Estrategia

In [96]:
current_price = df['base_price'].iloc[-1]
nearest_support = min(summary['support_levels'], key=lambda x: abs(x['mean_price']-current_price))

if current_price < nearest_support['mean_price'] * 1.01:  # 1% sobre el soporte
    print("Posible rebote técnico")

## Generando series de S&R y Market Imbalance

In [97]:
sr = []
mi = []
miu = []
mid = []
for i,row in df.iterrows():
    tot_vol = row['total_bid_volume']+row['total_ask_volume']
    for r in row['bid_supports']['levels']:
        if r['price']>row['base_price']*0.8:
            sr.append({'timestamp': row['timestamp'],
                    'price': r['price'],
                    'pct': round((r['volume']/tot_vol)*100,0),
                    })
    for r in row['ask_resistances']['levels']:
        if r['price']<row['base_price']*1.2:
            sr.append({'timestamp': row['timestamp'],
                    'price': r['price'],
                    'pct': round((r['volume']/tot_vol)*100,0),
                    })
    mi.append({'timestamp':row['timestamp'],
               'mi':-round(row['market_imbalance']['imbalance_pct'],2)})
    if row['market_imbalance']['imbalance_pct']>0:
        miu.append({'timestamp':row['timestamp'],
                    'mi':-round(row['market_imbalance']['imbalance_pct'],2)})
    if row['market_imbalance']['imbalance_pct']<0:
        mid.append({'timestamp':row['timestamp'],
                    'mi':-round(row['market_imbalance']['imbalance_pct'],2)})
    
sr = pd.DataFrame(sr)
sr_pct_max = sr['pct'].max()
sr_pct_min = sr['pct'].min()
sr['pct_adj'] = fn.map(sr['pct'],sr_pct_min,sr_pct_max,1,12)

mi = pd.DataFrame(mi, columns=['timestamp','mi'])
mi['ma'] = mi['mi'].rolling(window=48).mean()
miu = pd.DataFrame(miu, columns=['timestamp','mi'])
mid = pd.DataFrame(mid, columns=['timestamp','mi'])

## Creando Chart

In [98]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

chart_rows = 2
row_heights=[600,200]
total_height = sum(row_heights)

fig = make_subplots(rows=chart_rows, 
                    shared_xaxes=True,
                    row_heights=row_heights,
                    )

fig.add_trace(
        go.Scatter(
            x=df["timestamp"], y=df["base_price"], name=f'{symbol}', mode="lines",  
            line={'width': 0.75},  
            marker=dict(color='yellow',),
            legendgroup = '1',
            hovertemplate="%{x}<br>%{y}<extra></extra>",
        ),
        row=1,
        col=1,
    )  


fig.add_trace(
        go.Scatter(x=sr["timestamp"], y=sr['price'], name='Sop y Res', mode='markers', 
                marker=dict(symbol='circle',
                            size=sr['pct_adj'],
                            color=sr['pct_adj'],
                            colorscale="Aggrnyl",
                            #showscale=True,
                            line=dict(width=0,),
                            ),
                legendgroup = '1',
                hovertemplate="%{x}<br>%{y}<extra></extra>",
                ),row=1,col=1,
)

fig.add_trace(
        go.Bar(
            x=miu["timestamp"], y=miu["mi"], name=f'{symbol}',  
            marker=dict(color='#f6465d',
                        line=dict(width=0,)
                        ),
            hovertemplate="%{x}<br>%{y} %<extra></extra>",
            legendgroup = '2',
            showlegend=False,
        ),
        row=2,
        col=1,
    )  
fig.add_trace(
        go.Bar(
            x=mid["timestamp"], y=mid["mi"], name=f'{symbol}',  
            marker=dict(color='#0ecb81',
                        line=dict(width=0,)
                        ),
            hovertemplate="%{x}<br>%{y} %<extra></extra>",
            legendgroup = '2',
            showlegend=False,
        ),
        row=2,
        col=1,
    ) 

fig.add_trace(
        go.Scatter(
            x=mi["timestamp"], y=mi["ma"], name=f'MI MA', mode="lines",  
            line={'width': 0.75},  
            marker=dict(color='white',),
            legendgroup = '1',
            hovertemplate="%{x}<br>%{y}<extra></extra>",
        ),
        row=2,
        col=1,
    ) 



# Adjust layout for subplots
fig.update_layout(
    font=dict(color="#ffffff", family="Helvetica"),
    paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
    plot_bgcolor="#162024",  
    height=total_height,
    xaxis_rangeslider_visible=False,
    modebar_bgcolor="rgba(0,0,0,0)",
    legend_tracegroupgap = 100,
)

#Ajustar el tamaño de cada sub_plot
fig.update_layout(
    yaxis1=dict(title="Precio",showticklabels=True,),
    yaxis2=dict(title="Balance del Mercado (%)",showticklabels=True,),
)
fig.update_xaxes(showline=False, linewidth=2,linecolor='#000000', gridcolor='rgba(0,0,0,0)')
fig.update_yaxes(showline=False, linewidth=2, zeroline= False, linecolor='#ff0000', gridcolor='rgba(50,50,50,255)')     
chart = fig.show(config = {'scrollZoom': True, })

## Resample del DataFrame

In [ ]:
from datetime import timedelta

resample_periods = 2
resample_str = '1H'


df_indexado = df.set_index('timestamp')
dfr = df_indexado.resample(resample_str).first()
dfr = dfr.reset_index()
dfr = dfr[['timestamp']]
dfr['timestamp'] = pd.to_datetime(dfr['timestamp'].dt.strftime('%Y-%m-%d %H:%M'))
dfr['next_ts'] = dfr['timestamp'].shift(-1)
dfr['next_ts'].fillna(dfr['timestamp'] + timedelta(days=2), inplace=True)

analizer = oba.OrderBookAnalyzer()
data = []
sr = []
for i,row in dfr.iterrows():
    start = row['timestamp']
    end = row['next_ts']
    dfp = df[(df['timestamp']>=start)&(df['timestamp']<end)] 
    qty_recs = dfp['timestamp'].count()
    
    analizer.load_data(dfp)
    summary = analizer.get_summary_stats(qty_recs)
    data_r = {
            'timestamp': row['timestamp'],
            'high_price': summary['high_price'],
            'low_price': summary['low_price'],
            'mi': -summary['mean_imbalance'],       
            }
        
    data.append(data_r)
    for i in summary['support_levels']:
        if i['mean_price']>summary['low_price']*0.6:
            sr.append({'timestamp': row['timestamp'],
                    'price': i['mean_price'],
                    'pct': i['mean_volume_pct'],
                    })
    for i in summary['resistance_levels']:
        if i['mean_price']<summary['low_price']*1.4:
            sr.append({'timestamp': row['timestamp'],
                    'price': i['mean_price'],
                    'pct': i['mean_volume_pct'],
                    })

dfr = pd.DataFrame(data)
dfr['mi_ma'] = dfr['mi'].rolling(window=int(48/resample_periods)).mean()
sr = pd.DataFrame(sr)
sr_pct_max = sr['pct'].max()
sr_pct_min = sr['pct'].min()
sr['pct_adj'] = fn.map(sr['pct'],sr_pct_min,sr_pct_max,1,10)


## Grafica del Resample

In [ ]:
chart_rows = 2
row_heights=[600,200]
total_height = sum(row_heights)


fig = make_subplots(rows=chart_rows, 
                    shared_xaxes=True,
                    row_heights=row_heights,
                    )

# Línea High
fig.add_trace(
    go.Scatter(
        x=dfr["timestamp"], y=dfr["high_price"], name=f'{symbol}', mode="lines",
        line={'width': 0.5, 'color': '#fd7e14'},
        marker=dict(color='#fd7e14',),
        hovertemplate="%{x}<br>%{y}<extra></extra>",
        showlegend=False,
    ),
    row=1,
    col=1,
)

# Línea Low con relleno hacia la línea High
fig.add_trace(
    go.Scatter(
        x=dfr["timestamp"], y=dfr["low_price"], name=f'{symbol}', mode="lines",
        line={'width': 0.5, 'color': '#fd7e14'},
        marker=dict(color='#fd7e14',),
        hovertemplate="%{x}<br>%{y}<extra></extra>",
        showlegend=False,
        fill='tonexty',  # Rellena hacia la siguiente traza en el eje y
        fillcolor='rgba(253, 126, 20, 0.75)', # Color de relleno blanco (opcional, por defecto es el color de la traza anterior)
    ),
    row=1,
    col=1,
)


fig.add_trace(
    go.Scatter(x=sr["timestamp"], y=sr['price'], name='Sop y Res', mode='markers', 
        marker=dict(symbol='circle',
                    size=sr['pct_adj'],
                    color=sr['pct_adj'],
                    colorscale="Aggrnyl",
                    #showscale=True,
                    line=dict(width=0,),
                    ),
        hovertemplate="%{x}<br>%{y}<extra></extra>",
        showlegend=False,
        ),row=1,col=1,
    )

def get_bar_color(value):
    if value == 0:
        return '#333333'  # Gris oscuro para 0
    elif value > 25:
        return '#0ecb81'
    elif value > 10:
        return 'rgba(14, 203, 129, 0.75)'
    elif value > 0:
        return 'rgba(14, 203, 129, 0.5)'
    elif value < -25:
        return '#f6465d'
    elif value < -10:
        return 'rgba(246, 70, 93, 0.75)'
    elif value < 0:
        return 'rgba(246, 70, 93, 0.5)'
    else: 
        return '#333333'

# Aplicar la función para obtener la lista de colores
bar_colors = dfr["mi"].apply(get_bar_color)

# Gráfica de barras con color variable
fig.add_trace(
    go.Bar(
        x=dfr["timestamp"], y=dfr["mi"], name=f'{symbol}',
        marker=dict(color=bar_colors,
                    line=dict(width=0,)
                    ),
        hovertemplate="%{x}<br>%{y}<extra></extra>",
        showlegend=False,
    ), row=2, col=1,
)

# Adjust layout for subplots
fig.update_layout(
    font=dict(color="#ffffff", family="Helvetica"),
    paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
    plot_bgcolor="#162024",  
    height=total_height,
    xaxis_rangeslider_visible=False,
    modebar_bgcolor="rgba(0,0,0,0)",
)

#Ajustar el tamaño de cada sub_plot
fig.update_layout(
    yaxis1=dict(title="Precio",showticklabels=True,),
    yaxis2=dict(title="Balance del Mercado (%)",showticklabels=True,),
)
fig.update_xaxes(showline=False, linewidth=2,linecolor='#000000', gridcolor='rgba(0,0,0,0)')
fig.update_yaxes(showline=False, linewidth=2, zeroline= False, linecolor='#ff0000', gridcolor='rgba(50,50,50,255)')     
chart = fig.show(config = {'scrollZoom': True, })